In [1]:
from __future__ import print_function

import os
import json
import shutil
import sh
import yaml
from itertools import islice

import os
import glob
import sh
import yaml

from gitenberg import metadata
import jinja2

from second_folio import (GITENBERG_DIR, 
                          all_repos, 
                          apply_to_repos, 
                          travis_template, 
                          latest_epub,
                          new_travis_template,
                          repo_version,
                          )

from github_settings import (username, password)

REPOS_LIST = "/Users/raymondyee/C/src/gitenberg/Second-Folio/list_of_repos.txt"
GITENBERG_DIR = "/Users/raymondyee/C/src/gitenberg/"

METADATA_DIR = "/Users/raymondyee/C/src/gitenberg-dev/giten_site/metadata"
COVERS_DATA = "/Users/raymondyee/C/src/gitenberg/Second-Folio/covers_data.json"

In [3]:
repos = all_repos
repos

['Adventures-of-Huckleberry-Finn_76',
 'Don-Quixote_996',
 'Dubliners_2814',
 'Jane-Eyre_1260',
 'Moby-Dick--Or-The-Whale_2701',
 'Narrative-of-the-Life-of-Frederick-Douglass-an-American-Slave_23',
 'Pride-and-Prejudice_1342',
 'The-Adventures-of-Sherlock-Holmes_1661',
 'The-Brothers-Karamazov_28054',
 'The-Time-Machine_35',
 'Frankenstein_84',
 'Middlemarch_145',
 'A-Tale-of-Two-Cities_98',
 'The-Call-of-the-Wild_215',
 'Crime-and-Punishment_2554',
 'The-Strange-Case-of-Dr.-Jekyll-and-Mr.-Hyde_42',
 'Dracula_345',
 'Flatland--A-Romance-of-Many-Dimensions--Illustrated-_201',
 'Household-Stories-by-the-Brothers-Grimm_19068',
 'Heart-of-Darkness_219',
 'A-Journey-into-the-Interior-of-the-Earth_3748',
 'Jude-the-Obscure_153',
 'King-Solomon-s-Mines_2166',
 'Little-Women_514',
 'Madame-Bovary_2413',
 'The-Life-and-Adventures-of-Robinson-Crusoe_521',
 'The-Awakening-and-Selected-Short-Stories_160',
 'The-Jungle_140',
 'The-Jungle-Book_236',
 'Metamorphosis_5200',
 'The-Picture-of-Dorian-Gra

In [5]:
list(apply_to_repos(latest_epub, repos=repos))

['https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/releases/download/0.1.0/Adventures-of-Huckleberry-Finn.epub',
 'https://github.com/GITenberg/Don-Quixote_996/releases/download/0.0.12/Don-Quixote.epub',
 'https://github.com/GITenberg/Dubliners_2814/releases/download/0.0.3/Dubliners.epub',
 'https://github.com/GITenberg/Jane-Eyre_1260/releases/download/0.0.4/Jane-Eyre-An-Autobiography.epub',
 'https://github.com/GITenberg/Moby-Dick--Or-The-Whale_2701/releases/download/0.0.3/Moby-Dick-Or-The-Whale.epub',
 'https://github.com/GITenberg/Narrative-of-the-Life-of-Frederick-Douglass-an-American-Slave_23/releases/download/0.0.3/Narrative-of-the-Life-of-Frederick-Douglass-an-American-Slave.epub',
 'https://github.com/GITenberg/Pride-and-Prejudice_1342/releases/download/0.0.2/Pride-and-Prejudice.epub',
 'https://github.com/GITenberg/The-Adventures-of-Sherlock-Holmes_1661/releases/download/0.0.2/The-Adventures-of-Sherlock-Holmes.epub',
 'https://github.com/GITenberg/The-Brothers-Ka

# robobrowser

In [76]:
from robobrowser import RoboBrowser

def post_to_yaml_loader(url, unglue_url="https://unglue.it/api/loader/yaml"):
        
    browser = RoboBrowser(history=True)
    browser.open(unglue_url)

    form = browser.get_forms()[0]
    form['repo_url'] = url
    # weird I have to manually set referer
    browser.session.headers['referer'] = unglue_url
    browser.submit_form(form)
    
    return browser

In [79]:
b = post_to_yaml_loader('https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/raw/master/metadata.yaml')
(b.url, b.response)

(u'https://unglue.it/work/155766/', <Response [200]>)

# opds

In [83]:
from lxml import etree
import requests

opds_url = "https://unglue.it/api/opds/"
doc = etree.fromstring(requests.get(opds_url).content)
doc

<Element {http://www.w3.org/2005/Atom}feed at 0x109dc3e60>

# Failed attempt with requests to submit to yaml loader

In [39]:
import requests

from lxml import etree
from lxml.cssselect import CSSSelector

unglue_url = "https://unglue.it/api/loader/yaml"

r = requests.get(unglue_url)
doc = etree.HTML(r.content)

sel = CSSSelector('input[name="csrfmiddlewaretoken"]')
csrftoken = sel(doc)[0].attrib.get('value')
csrftoken

'HegKNWTMjkRV6Wiv3OzPnq5UuTNqvjbW'

In [40]:
r = requests.post(unglue_url, 
                  data={'repo_url':
                          'https://github.com/GITenberg/Adventures-of-Huckleberry-Finn_76/raw/master/metadata.yaml',
                         'csrfmiddlewaretoken':csrftoken
                         },
                  headers={'referer':unglue_url})

In [41]:
(r.status_code, r.content)

(403,
 '\n<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8">\n  <meta name="robots" content="NONE,NOARCHIVE">\n  <title>403 Forbidden</title>\n  <style type="text/css">\n    html * { padding:0; margin:0; }\n    body * { padding:10px 20px; }\n    body * * { padding:0; }\n    body { font:small sans-serif; background:#eee; }\n    body>div { border-bottom:1px solid #ddd; }\n    h1 { font-weight:normal; margin-bottom:.4em; }\n    h1 span { font-size:60%; color:#666; font-weight:normal; }\n    #info { background:#f6f6f6; }\n    #info ul { margin: 0.5em 4em; }\n    #info p, #summary p { padding-top:10px; }\n    #summary { background: #ffc; }\n    #explanation { background:#eee; border-bottom: 0px none; }\n  </style>\n</head>\n<body>\n<div id="summary">\n  <h1>Forbidden <span>(403)</span></h1>\n  <p>CSRF verification failed. Request aborted.</p>\n\n</div>\n\n<div id="explanation">\n  <p><small>More information is available with DEBU